In [15]:
import matplotlib.pyplot as plt

import torch

from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"

from diffusion_framework.ddpm_1d import DDPM_1d
from diffusion_framework.nets import ErrorNet, CondErrorNet

from torch.optim import Adam
import numpy as np

import os
import gym
import numpy as np
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
import platform
if platform.system() == "Linux":
    os.environ['PYOPENGL_PLATFORM'] = 'egl'

### Select environment
env_id = "MountainCarContinuous-v0"
# Note that the algorithm is SAC

gym_env = gym.make(env_id)
max_obs_values = gym_env.observation_space.high
min_obs_values = gym_env.observation_space.low

max_act_value = gym_env.action_space.high
min_act_value = gym_env.action_space.low

bias = max_obs_values + min_obs_values
bias = bias / 2
scale = max_obs_values - min_obs_values
scale = scale / 2

env = make_vec_env(env_id, n_envs=1)
best_model = SAC.load('/home/sai-admin/advanced_ml_project/rl-baselines3-zoo/rl-trained-agents/sac/' + 
                      env_id + '_1/' + 
                      env_id + '.zip', env=env)

In [36]:
# Define data
expert_demo_path = 'expert/logs/expert_demonstrations.npy'

dataset = np.load(expert_demo_path, allow_pickle=True)
dataset = torch.tensor(dataset, dtype=torch.float32)

batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define models
timesteps = 100
diffusion = DDPM_1d(timesteps)

In [38]:
# Visualizing corruption process

# x_sample = dataset[:10000].unsqueeze(0)
# t = torch.tensor([0,250,500,750,999])
# out = diffusion.q_sample(x_sample, t)
# N = 5
# fig, ax = plt.subplots(1,5,figsize=(N*2,2))
# for i in range(N):
#     ax[i].scatter(out[i, :, 0], out[i, :, 1], s=2)
#     ax[i].set_xlim(-1,1)
#     ax[i].set_ylim(-1,1)
# plt.tight_layout()

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = CondErrorNet(dim=1,cond_dim=2)
model.to(device)

optimizer = Adam(model.parameters(), lr=1e-4)
model

cuda:0


CondErrorNet(
  (time_mlp): Sequential(
    (0): SinusoidalPositionEmbeddings()
    (1): Linear(in_features=64, out_features=128, bias=True)
    (2): GELU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): SiLU()
    (5): Linear(in_features=64, out_features=2, bias=True)
  )
  (state_mlp): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): GELU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): GELU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): GELU()
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
  (res_mlp): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): SiLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
  (final_mlp): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): GELU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): GELU()
    (4): Linear(in_features=128, out_featu

In [40]:
epochs = 20

for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
        optimizer.zero_grad()

        batch_size = batch.shape[0]
        batch = batch.to(device)

        # Algorithm 1 line 3: sample t uniformally for every example in the batch
        t = torch.randint(0, timesteps, (batch_size,), device=device).long()

        loss = diffusion.conditional_p_losses(model, batch, t, loss_type="huber")

        if step % 100 == 0:
            print("Epoch: %d, Loss: %f" %(epoch, loss.item()))

        loss.backward()
        optimizer.step()


Epoch: 0, Loss: 0.430114
Epoch: 0, Loss: 0.357732
Epoch: 0, Loss: 0.217975
Epoch: 0, Loss: 0.145122
Epoch: 0, Loss: 0.206579
Epoch: 0, Loss: 0.163589
Epoch: 0, Loss: 0.171231
Epoch: 0, Loss: 0.157102
Epoch: 0, Loss: 0.162236
Epoch: 0, Loss: 0.099001
Epoch: 0, Loss: 0.136750
Epoch: 0, Loss: 0.123100
Epoch: 0, Loss: 0.086983
Epoch: 0, Loss: 0.135005
Epoch: 0, Loss: 0.161553
Epoch: 0, Loss: 0.130541
Epoch: 0, Loss: 0.114306
Epoch: 0, Loss: 0.095399
Epoch: 0, Loss: 0.103303
Epoch: 0, Loss: 0.129993
Epoch: 0, Loss: 0.109435
Epoch: 0, Loss: 0.077038
Epoch: 0, Loss: 0.091411
Epoch: 0, Loss: 0.101343
Epoch: 0, Loss: 0.076956
Epoch: 0, Loss: 0.102985
Epoch: 0, Loss: 0.083000
Epoch: 0, Loss: 0.094485
Epoch: 0, Loss: 0.099095
Epoch: 0, Loss: 0.065592
Epoch: 0, Loss: 0.060605
Epoch: 0, Loss: 0.087256
Epoch: 0, Loss: 0.074713
Epoch: 0, Loss: 0.057595
Epoch: 0, Loss: 0.066759
Epoch: 0, Loss: 0.045417
Epoch: 0, Loss: 0.055380
Epoch: 0, Loss: 0.056336
Epoch: 0, Loss: 0.050846
Epoch: 0, Loss: 0.055994


Epoch: 4, Loss: 0.013671
Epoch: 4, Loss: 0.016053
Epoch: 4, Loss: 0.025013
Epoch: 4, Loss: 0.028327
Epoch: 4, Loss: 0.020766
Epoch: 4, Loss: 0.012013
Epoch: 4, Loss: 0.017137
Epoch: 4, Loss: 0.027727
Epoch: 4, Loss: 0.013306
Epoch: 4, Loss: 0.013166
Epoch: 4, Loss: 0.010040
Epoch: 4, Loss: 0.032870
Epoch: 4, Loss: 0.010741
Epoch: 4, Loss: 0.016809
Epoch: 4, Loss: 0.034235
Epoch: 4, Loss: 0.021551
Epoch: 4, Loss: 0.010542
Epoch: 4, Loss: 0.025413
Epoch: 4, Loss: 0.014254
Epoch: 4, Loss: 0.019865
Epoch: 4, Loss: 0.051387
Epoch: 4, Loss: 0.019933
Epoch: 4, Loss: 0.013534
Epoch: 4, Loss: 0.022058
Epoch: 4, Loss: 0.013166
Epoch: 4, Loss: 0.027430
Epoch: 4, Loss: 0.021431
Epoch: 4, Loss: 0.019225
Epoch: 4, Loss: 0.016859
Epoch: 4, Loss: 0.015152
Epoch: 4, Loss: 0.008568
Epoch: 4, Loss: 0.021791
Epoch: 4, Loss: 0.015998
Epoch: 4, Loss: 0.021424
Epoch: 4, Loss: 0.015685
Epoch: 4, Loss: 0.024452
Epoch: 4, Loss: 0.010634
Epoch: 4, Loss: 0.010325
Epoch: 4, Loss: 0.027455
Epoch: 4, Loss: 0.023856


Epoch: 8, Loss: 0.023989
Epoch: 8, Loss: 0.034135
Epoch: 8, Loss: 0.025515
Epoch: 8, Loss: 0.005622
Epoch: 8, Loss: 0.016998
Epoch: 8, Loss: 0.004387
Epoch: 8, Loss: 0.003562
Epoch: 8, Loss: 0.015082
Epoch: 8, Loss: 0.019754
Epoch: 8, Loss: 0.008440
Epoch: 8, Loss: 0.012654
Epoch: 8, Loss: 0.014601
Epoch: 8, Loss: 0.004279
Epoch: 8, Loss: 0.008896
Epoch: 8, Loss: 0.009156
Epoch: 8, Loss: 0.011961
Epoch: 8, Loss: 0.005751
Epoch: 8, Loss: 0.015179
Epoch: 8, Loss: 0.023014
Epoch: 8, Loss: 0.008555
Epoch: 8, Loss: 0.010561
Epoch: 8, Loss: 0.005550
Epoch: 8, Loss: 0.016098
Epoch: 8, Loss: 0.011278
Epoch: 8, Loss: 0.008379
Epoch: 8, Loss: 0.013069
Epoch: 8, Loss: 0.021893
Epoch: 8, Loss: 0.014761
Epoch: 8, Loss: 0.012266
Epoch: 8, Loss: 0.016863
Epoch: 8, Loss: 0.009229
Epoch: 8, Loss: 0.009660
Epoch: 8, Loss: 0.010422
Epoch: 8, Loss: 0.011213
Epoch: 8, Loss: 0.013431
Epoch: 8, Loss: 0.016112
Epoch: 8, Loss: 0.009798
Epoch: 8, Loss: 0.025606
Epoch: 8, Loss: 0.010105
Epoch: 8, Loss: 0.011880


Epoch: 12, Loss: 0.010170
Epoch: 12, Loss: 0.005123
Epoch: 12, Loss: 0.008568
Epoch: 12, Loss: 0.012900
Epoch: 12, Loss: 0.009857
Epoch: 12, Loss: 0.007693
Epoch: 12, Loss: 0.007165
Epoch: 12, Loss: 0.004209
Epoch: 12, Loss: 0.012198
Epoch: 12, Loss: 0.007672
Epoch: 12, Loss: 0.014557
Epoch: 12, Loss: 0.012632
Epoch: 12, Loss: 0.008377
Epoch: 12, Loss: 0.005519
Epoch: 12, Loss: 0.013161
Epoch: 12, Loss: 0.012123
Epoch: 12, Loss: 0.011639
Epoch: 12, Loss: 0.004345
Epoch: 12, Loss: 0.003349
Epoch: 12, Loss: 0.008645
Epoch: 12, Loss: 0.008030
Epoch: 12, Loss: 0.014844
Epoch: 12, Loss: 0.010861
Epoch: 12, Loss: 0.009066
Epoch: 12, Loss: 0.013532
Epoch: 12, Loss: 0.007548
Epoch: 12, Loss: 0.012695
Epoch: 12, Loss: 0.003910
Epoch: 12, Loss: 0.005475
Epoch: 12, Loss: 0.005302
Epoch: 12, Loss: 0.006338
Epoch: 12, Loss: 0.019589
Epoch: 12, Loss: 0.004839
Epoch: 12, Loss: 0.021625
Epoch: 12, Loss: 0.013311
Epoch: 12, Loss: 0.022571
Epoch: 12, Loss: 0.002953
Epoch: 12, Loss: 0.014674
Epoch: 12, L

Epoch: 16, Loss: 0.005633
Epoch: 16, Loss: 0.009408
Epoch: 16, Loss: 0.008184
Epoch: 16, Loss: 0.006743
Epoch: 16, Loss: 0.007087
Epoch: 16, Loss: 0.013512
Epoch: 16, Loss: 0.014774
Epoch: 16, Loss: 0.003035
Epoch: 16, Loss: 0.008752
Epoch: 16, Loss: 0.006441
Epoch: 16, Loss: 0.007314
Epoch: 16, Loss: 0.003435
Epoch: 16, Loss: 0.006320
Epoch: 16, Loss: 0.007881
Epoch: 16, Loss: 0.006892
Epoch: 16, Loss: 0.004675
Epoch: 16, Loss: 0.004861
Epoch: 16, Loss: 0.004176
Epoch: 16, Loss: 0.005443
Epoch: 16, Loss: 0.007656
Epoch: 16, Loss: 0.004299
Epoch: 16, Loss: 0.012046
Epoch: 16, Loss: 0.012885
Epoch: 16, Loss: 0.009662
Epoch: 16, Loss: 0.010548
Epoch: 16, Loss: 0.003884
Epoch: 16, Loss: 0.006001
Epoch: 16, Loss: 0.009561
Epoch: 16, Loss: 0.019247
Epoch: 16, Loss: 0.019203
Epoch: 16, Loss: 0.013306
Epoch: 16, Loss: 0.003682
Epoch: 16, Loss: 0.004303
Epoch: 16, Loss: 0.005715
Epoch: 16, Loss: 0.016024
Epoch: 16, Loss: 0.005089
Epoch: 16, Loss: 0.007069
Epoch: 16, Loss: 0.010628
Epoch: 16, L

In [6]:
for step, batch in enumerate(dataloader): break
cond = batch[:, :-1].to(device) 
samples = diffusion.conditional_sample(cond, model, image_size=1, batch_size=128)

sampling loop time step:   0%|          | 0/10000 [00:00<?, ?it/s]

In [7]:
cond = cond.detach().cpu().numpy()
trial_samples = cond * scale + bias
best_model.predict(trial_samples)[0]

array([[-0.842015  ],
       [-0.8195985 ],
       [ 0.98319817],
       [ 0.9791057 ],
       [-0.42925155],
       [-0.03757697],
       [ 0.14848042],
       [ 0.41027284],
       [ 0.8430635 ],
       [-0.45497656],
       [-0.05592513],
       [ 0.9832127 ],
       [ 0.98097944],
       [ 0.9548286 ],
       [-0.21146631],
       [-0.25092345],
       [ 0.34486234],
       [-0.38975686],
       [ 0.98119617],
       [ 0.82282114],
       [ 0.9799963 ],
       [-0.34440982],
       [ 0.98209167],
       [ 0.5164199 ],
       [ 0.5192437 ],
       [ 0.44996905],
       [ 0.8777826 ],
       [ 0.9787164 ],
       [-0.21505892],
       [-0.12742108],
       [ 0.95991135],
       [-0.3614183 ],
       [-0.3639124 ],
       [ 0.2473414 ],
       [ 0.98543215],
       [-0.19490314],
       [-0.12956715],
       [ 0.7173605 ],
       [ 0.9825808 ],
       [ 0.622939  ],
       [-0.17044902],
       [ 0.95837593],
       [-0.0820961 ],
       [-0.13803995],
       [ 0.98026514],
       [ 0

In [41]:
for i in range(1):
    obs = env.reset()
    print(obs.shape)
    done = False 
    while not done:
        #action, _= best_model.predict(obs)
        print(obs)
        cond = torch.tensor((obs-bias)/scale).to(device)
        samples = diffusion.conditional_sample(cond, model, image_size=1, batch_size=1)
        action = samples[-1]
        obs, reward, done, info = env.step(action)
        if done:
            print("reward at the end of the episode : ", reward)

(1, 2)
[[-0.56191593  0.        ]]


sampling loop time step:   0%|          | 0/100 [00:00<?, ?it/s]

[[-5.616663e-01  2.496418e-04]]


sampling loop time step:   0%|          | 0/100 [00:00<?, ?it/s]

[[-5.6114668e-01  5.1966263e-04]]


...

[[0.4064931  0.02791713]]


sampling loop time step:   0%|          | 0/100 [00:00<?, ?it/s]

[[0.4350193  0.02852623]]


sampling loop time step:   0%|          | 0/100 [00:00<?, ?it/s]

reward at the end of the episode :  [99.90146]


In [21]:
cond = torch.tensor(obs).to(device)

/tmp/ipykernel_1003954/3536313676.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cond = torch.tensor(obs).to(device)


In [22]:
samples = diffusion.conditional_sample(cond, model, image_size=1, batch_size=1)

sampling loop time step:   0%|          | 0/10000 [00:00<?, ?it/s]

In [27]:
samples[-1].detach()

AttributeError: 'numpy.ndarray' object has no attribute 'detach'